In [1]:
import tigercontrol
from tigercontrol.experiments import Experiment
from tigercontrol.methods.optimizers import *

/Users/paulag/anaconda3/lib/python3.6/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
exp = Experiment()
exp.initialize(timesteps = 100)
exp.add_problem('MyProblem-v0', {'file' : 'data/enso.csv'})
exp.add_method('LSTM', {'l' : 32, 'h' : 64, 'optimizer' : OGD}, name = '32-64-OGD')
exp.add_method('LSTM', {'l' : 32, 'h' : 64, 'optimizer' : Adagrad}, name = '32-64-Adagrad')
exp.add_method('LSTM', {'l' : 32, 'h' : 64,  'optimizer' : Adam}, name = '32-64-Adam')

exp.add_method('SimpleBoost', {'method_id': 'LSTM', 'method_params': {'optimizer' : OGD}}, name = 'LSTM-OGD')
#exp.add_method('SimpleBoost', {'method_id': 'LSTM', \
#    'method_params': {'n' : 1, 'm' : 1, 'optimizer' : Adagrad}}, name = 'LSTM-Adagrad')
#exp.add_method('SimpleBoost', {'method_id': 'LSTM', \
#    'method_params': {'n' : 1, 'm' : 1, 'optimizer' : Adam}}, name = 'LSTM-Adam')

In [3]:
exp.scoreboard()

Average mse:
+----------------------+-----------+
|   Method\Problems    | MyPr..em- |
+----------------------+-----------+
|    LSTM-32-64-OGD    |    nan    |
|  LSTM-32-64-Adagrad  |    nan    |
|   LSTM-32-64-Adam    |    nan    |
| SimpleBoost-LSTM-OGD |    nan    |
+----------------------+-----------+


In [ ]:
exp.graph(yscale = 'log')